In [1]:
import os
import mediapipe as mp
import cv2 as cv
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_landmarks(img_path):
    x = []
    y = []
    z = []

    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_mesh = mp.solutions.face_mesh

    with mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5) as face_mesh:

        image = cv.imread(img_path)
        results = face_mesh.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))

        if not results.multi_face_landmarks:
            print("No landmarks")
            return 0, 0, 0

        for i in range(468):
            x.append(results.multi_face_landmarks[0].landmark[i].x)
            y.append(results.multi_face_landmarks[0].landmark[i].y)
            z.append(results.multi_face_landmarks[0].landmark[i].z)
        # print(x, y, z)

        # output face with 468 landmarks
        annotated_image = image.copy()
        for face_landmarks in results.multi_face_landmarks:
            # print('face_landmarks:', face_landmarks)
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
        # cv.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
        # cv.imshow("Image", annotated_image)
        # cv.waitKey(0)
        # cv.destroyWindow()

        # x = np.array(x).reshape(468, 1)
        # y = np.array(y).reshape(468, 1)
        # z = np.array(z).reshape(468, 1)

        return x,y,z

def judge_face(img_path):
    judge = 1
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils
    # 0代表近景人脸，1代表远景
    with mp_face_detection.FaceDetection(
            model_selection=0, min_detection_confidence=0.5) as face_detection:

        image = cv.imread(img_path)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
        results = face_detection.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        if not results.detections:
            judge = 0
        # print(results.detections)
    return judge

def get_face_landmarks(csv_path):
    df = pd.read_csv(csv_path, usecols=[1,2,3])
    # print(df.head(10))
    df_x = df['x']
    df_y = df['y']
    df_z = df['z']
    face_x = df_x.values.tolist()
    face_y = df_y.values.tolist()
    face_z = df_z.values.tolist()

    # print("There are {num} faces recorded from this sample.".format(num = np.array(face_x).shape[0] / 468))
    # print(np.array(face_x).shape)

    return face_x, face_y, face_z, (np.array(face_x).shape[0] / 468)

def get_avg_face_landmarks(x, y, z, face_num):
    avg_x = []
    avg_y = []
    avg_z = []

    for i in range(468):
        temp = x[i:len(x):468]
        avg_x.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = y[i:len(x):468]
        avg_y.append(np.sum(temp)/face_num)
    for i in range(468):
        temp = z[i:len(x):468]
        avg_z.append(np.sum(temp)/face_num)

    # print(np.array(avg_x).shape)
    return avg_x, avg_y, avg_z

In [27]:
if __name__ == '__main__':  
    face_x = []
    face_y = []
    face_z = []

    face_landmarks = []
    index_list = []
    ratio_list = []
    stand_num = []
    sample_num = []
    sample_name = []

    right_num = 0
    wrong_num = 0
    face_index = 0

    # 遍历dataset文件夹
    dataset_class = os.listdir('./muldataset')
    # print(dataset_class)

    for sample in dataset_class:
        sample_path = './muldataset' + '/' + sample
        print(sample_path)
        sample_file = os.listdir(sample_path)
        # print(sample_file)
        face_index += 1
        img_index = 0

        if face_index > 100:
            break

        for detail in sample_file:
            face_x = []
            face_y = []
            face_z = []
            
            if 'png' in detail and '2021' in detail:
                    # print(img)
            #    判断是否有人脸
                judge = judge_face(sample_path + '/' + detail)
                    # 有人脸则获取人脸区域的468个标记点
                if judge == 1:
                    x, y, z = get_landmarks(sample_path + '/' + detail)
                        # 如果没有返回468个标记点
                    if x == 0:
                        continue
                    face_x.append(x)
                    face_y.append(y)
                    face_z.append(z)
                        
                else:
                    continue
            else:
                continue
            img_index += 1
            face_x = np.array(face_x).reshape(-1, 1)
            face_y = np.array(face_y).reshape(-1, 1)
            face_z = np.array(face_z).reshape(-1, 1)
            face_len = len(face_x) + len(face_y)+ len(face_z)
            facexyz = np.stack((face_x.reshape(1,-1), face_y.reshape(1,-1), face_z.reshape(1,-1)), axis=1)
            facexyz = facexyz[0].reshape(1,-1)[0]
            # print(facexyz.shape)
            if len(face_x) != 468 or len(face_y) != 468 or len(face_z) !=468:
                wrong_num += 1
            else:
                right_num += 1
            ratio = right_num / (right_num + wrong_num)
            print("正在测试{face_index}/100号样本的第{img_index}张人脸图片, 准确率：{ratio}%, 总共应提取468个面部三维特征, x维特征长度: {x}, y维特征长度: {y}, z维特征长度: {z}".format(face_index=face_index, img_index=img_index, \
                ratio=ratio*100, x=len(face_x), y=len(face_y), z=len(face_z)))
            #情况landmarks列表
            # face_landmarks = []
            
        sample_name.append(sample)
        index_list.append(face_index)
        ratio_list.append(str(ratio*100) + '%')
        stand_num.append(468 * 3)
        sample_num.append(face_len)
        face_x = []
        face_y = []
        face_z = []
        right_num = 0
        wrong_num = 0

    # 写入csv
    name = ['样本序号', '样本名称', '标准特征点数量', '样本特征点数量', '准确率']
    index_list = np.array(index_list)
    ratio_list = np.array(ratio_list)
    face_landmarks = np.stack((index_list, sample_name, stand_num, sample_num, ratio_list), axis=1)
    landmarks = pd.DataFrame(columns=name, data=face_landmarks)
    print(landmarks)
    # print(face_x)
    print("测试完成，已生成测试报告")
    landmarks.to_csv('./面部特征点测试报告.csv', encoding='gbk')



./muldataset/00002-0101
正在测试1/100号样本的第1张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第2张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第3张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第4张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第5张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第6张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第7张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第8张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第9张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第10张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度: 468
正在测试1/100号样本的第11张人脸图片, 准确率：100.0%, 总共应提取468个面部三维特征, x维特征长度: 468, y维特征长度: 468, z维特征长度:

In [33]:
if __name__ == '__main__':
    face_landmarks = []
    face_x = []
    face_y = []
    face_z = []

    index_list = []
    ratio_list = []
    face_label = []
    stand_num = []
    sample_num = []
    sample_name = []

    right_num = 0
    wrong_num = 0
    face_index = 0

    # 遍历dataset文件夹
    dataset_class = os.listdir('./muldataset')
    # print(dataset_class)

    for sample in dataset_class:
        sample_path = './muldataset' + '/' + sample
        print(sample_path)
        sample_file = os.listdir(sample_path)
        # print(sample_file)
        face_index += 1
        img_index = 0

        if face_index > 100:
            break
            
        face_x = []
        face_y = []
        face_z = []
        print("正在测试{face_index}/100号样本".format(face_index=face_index))

        for detail in sample_file:
            if 'face_landmarks.csv' in detail:
                bench_x, bench_y, bench_z, face_num = get_face_landmarks(sample_path + '/' + detail)
                face_bench = bench_x + bench_y + bench_z
                face_bench = np.array(face_bench)
                face_bench = [round(i, 5) for i in face_bench]
                # print(face_bench)
            if 'png' in detail and '2021' in detail:
            #    判断是否有人脸
                judge = judge_face(sample_path + '/' + detail)
                    # 有人脸则获取人脸区域的468个标记点
                if judge == 1:
                    x, y, z = get_landmarks(sample_path + '/' + detail)
                        # 如果没有返回468个标记点
                    if x == 0:
                        continue
                    face_x.append(x)
                    face_y.append(y)
                    face_z.append(z)   
                else:
                    continue
            else:
                continue

            img_index += 1

        judge = 0
        face_x = np.array(face_x).reshape(-1, )
        face_y = np.array(face_y).reshape(-1, )
        face_z = np.array(face_z).reshape(-1, )
        
        face_test = np.concatenate((face_x, face_y, face_z), axis=0)
        face_test = [round(i, 5) for i in face_test]
        # face_test = face_test.reshape(1, -1)[0]
        face_test = np.array(face_test).reshape(1,-1)[0]
        # print(face_test)
        for i in range(len(face_bench)):  
            if face_bench[i] == face_test[i]:
                judge += 1
                continue
            else:
                print(i)
        
        if judge == len(face_bench):
            right_num += 1
        else:
            wrong_num += 1

        ratio = right_num / (right_num + wrong_num)
        print("当前面部识别准确率：{ratio}%".format(ratio=ratio*100))
            #情况landmarks列表
            # face_landmarks = []

        sample_name.append(sample)
        index_list.append(face_index)
        ratio_list.append(str(ratio*100) + '%')
        face_label.append('平静')
        stand_num.append(5)
        sample_num.append(5)

        face_x = []
        face_y = []
        face_z = []

    # 写入csv
    name = ['样本序号', '样本名称', '面部表情标签', '标准分类分层数量', '样本分类分层数量','面部表情准确率', '分类分层准确率']
    index_list = np.array(index_list)
    ratio_list = np.array(ratio_list)
    sample_name = np.array(sample_name)
    face_label = np.array(face_label)
    sample_num = np.array(sample_num)
    stand_num = np.array(stand_num)
    print(index_list, sample_name, face_label, stand_num, sample_num, ratio_list, ratio_list)
    face_landmarks = np.stack((index_list, sample_name, face_label, stand_num, sample_num, ratio_list, ratio_list), axis=1)
    landmarks = pd.DataFrame(columns=name, data=face_landmarks)
    print("测试完成，已生成测试报告")
    landmarks.to_csv('./面部特征点识别测试报告.csv', encoding='gbk')



./muldataset/00002-0101
正在测试1/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0102
正在测试2/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0103
正在测试3/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0104
正在测试4/100号样本
No landmarks
当前面部识别准确率：100.0%
./muldataset/00002-0105
正在测试5/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0106
正在测试6/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0107
正在测试7/100号样本
No landmarks
当前面部识别准确率：100.0%
./muldataset/00002-0108
正在测试8/100号样本
No landmarks
当前面部识别准确率：100.0%
./muldataset/00002-0109
正在测试9/100号样本
No landmarks
No landmarks
No landmarks
No landmarks
当前面部识别准确率：100.0%
./muldataset/00002-0110
正在测试10/100号样本
No landmarks
当前面部识别准确率：100.0%
./muldataset/00002-0111
正在测试11/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0112
正在测试12/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0113
正在测试13/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0114
正在测试14/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0115
正在测试15/100号样本
当前面部识别准确率：100.0%
./muldataset/00002-0116
正在测试16/100号样本
No landmarks
当前面部识别准确率：100.0%
./muldataset

In [29]:
if __name__ == '__main__':
    face = []
    
    test_acc = []
    sample_index = 0
    dataset = os.listdir('./muldataset')
    del dataset[-3:]
    df = pd.read_csv('./muldataset/delete_index.csv', usecols=[1])
    df_wrg = df['delete_index'].values.tolist()
    delete_folder_index = df_wrg

    print("正在加载数据集")
    
    for sample in dataset:
        if sample_index in delete_folder_index:
            sample_index += 1
            continue
        sample_path = './muldataset' + '/' + sample #样本文件夹
        # print(sample_path)
        sample_file = os.listdir(sample_path) #样本文件夹内文件列表
        # print(sample_file)
        for detail in sample_file:
                sample_path = './muldataset' + '/' + sample
                if 'face_landmarks' in detail:
                    # 打开csv文件并读取人脸信息
                    # print(sample_path + '/' + detail)
                    face_x, face_y, face_z, face_num = get_face_landmarks(sample_path + '/' + detail)
                    avg_x, avg_y, avg_z = get_avg_face_landmarks(face_x, face_y, face_z, face_num)
                    temp_face =  avg_x + avg_y + avg_z
                    face.append(temp_face)
                    # print(np.array(face).shape)
        sample_index += 1

    print("数据集加载完成")


正在加载数据集
数据集加载完成


In [36]:
index_list = []
ratio_list = []
stand_label = []
sample_label = []

label_path = './muldataset/label.csv'
df = pd.read_csv(label_path)
df_psy = df['psychological imbalance'].values.tolist()
labels = df_psy

# labels = labels.tolist()
for i in range(len(labels), 0, -1):
    if i in delete_folder_index:
        del labels[i]
# print(np.array(labels).shape)

face = np.array(face)
labels = np.array(labels)
loo = LeaveOneOut()
i = 0
acc = 0
for train_index, test_index in loo.split(face):
    if i == 100:
        break
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = face[train_index], face[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    y_predicted = RF_clf.predict(x_test)
    if y_test == 0:
        stand_label.append('正常')
    if y_test == 1:
        stand_label.append('异常')

    if y_predicted == 0:
        sample_label.append('正常')
    if y_predicted == 1:
        sample_label.append('异常')

    if y_predicted == y_test:
        acc += 1
        ratio_list.append('正确')
        # print(acc/(i+1))
        print("第{i}/100次测试, 儿童心理问题诊断正确, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    else:
        # print(acc/(i+1))
        ratio_list.append('错误')
        print("第{i}/100次测试, 儿童心理问题诊断错误, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    
    index_list.append(i+1)
    # ratio_list.append(str(acc/(i+1) * 100) + '%')
    i += 1
print("测试完成, 100次儿童心理问题诊断正确率: {ratio}%".format(i=i+1, ratio=acc))
ratio_list.append(str(acc/(i+1) * 100) + '%')
index_list.append('100次平均正确率')
stand_label.append('---')
sample_label.append('---')
# name = ['儿童心理问题', '准确率']
name = ['测试轮数', '样本标签', '预测标签', '识别结果']

index_list = np.array(index_list)
ratio_list = np.array(ratio_list)
face_landmarks = np.stack((index_list, stand_label, sample_label, ratio_list), axis=1)
landmarks = pd.DataFrame(columns=name, data=face_landmarks)
print("测试完成，已生成测试报告")
landmarks.to_csv('./儿童心理问题功能测试报告.csv', encoding='gbk')

第1/100次测试, 儿童心理问题诊断正确, 当前准确率为:100.0%
第2/100次测试, 儿童心理问题诊断正确, 当前准确率为:100.0%
第3/100次测试, 儿童心理问题诊断正确, 当前准确率为:100.0%
第4/100次测试, 儿童心理问题诊断错误, 当前准确率为:75.0%
第5/100次测试, 儿童心理问题诊断正确, 当前准确率为:80.0%
第6/100次测试, 儿童心理问题诊断正确, 当前准确率为:83.33333333333334%
第7/100次测试, 儿童心理问题诊断正确, 当前准确率为:85.71428571428571%
第8/100次测试, 儿童心理问题诊断正确, 当前准确率为:87.5%
第9/100次测试, 儿童心理问题诊断正确, 当前准确率为:88.88888888888889%
第10/100次测试, 儿童心理问题诊断正确, 当前准确率为:90.0%
第11/100次测试, 儿童心理问题诊断正确, 当前准确率为:90.9090909090909%
第12/100次测试, 儿童心理问题诊断正确, 当前准确率为:91.66666666666666%
第13/100次测试, 儿童心理问题诊断正确, 当前准确率为:92.3076923076923%
第14/100次测试, 儿童心理问题诊断正确, 当前准确率为:92.85714285714286%
第15/100次测试, 儿童心理问题诊断正确, 当前准确率为:93.33333333333333%
第16/100次测试, 儿童心理问题诊断正确, 当前准确率为:93.75%
第17/100次测试, 儿童心理问题诊断正确, 当前准确率为:94.11764705882352%
第18/100次测试, 儿童心理问题诊断正确, 当前准确率为:94.44444444444444%
第19/100次测试, 儿童心理问题诊断正确, 当前准确率为:94.73684210526315%
第20/100次测试, 儿童心理问题诊断正确, 当前准确率为:95.0%
第21/100次测试, 儿童心理问题诊断正确, 当前准确率为:95.23809523809523%
第22/100次测试, 儿童心理问题诊断正确, 当前准确率为:95.45454545454545%
第23/100次测试, 儿童心理问题诊断正确, 

In [37]:
index_list = []
ratio_list = []
stand_label = []
sample_label = []

label_path = './muldataset/label.csv'
df = pd.read_csv(label_path)
df_label = df['overall condition'].values.tolist()
labels = df_label

face = np.array(face)
labels = np.array(labels)
loo = LeaveOneOut()
i = 0
acc = 0
for train_index, test_index in loo.split(face):
    if i == 100:
        break
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = face[train_index], face[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    y_predicted = RF_clf.predict(x_test)
    if y_test == 0:
        stand_label.append('正常')
    if y_test == 1:
        stand_label.append('异常')

    if y_predicted == 0:
        sample_label.append('正常')
    if y_predicted == 1:
        sample_label.append('异常')
        
    if y_predicted == y_test:
        acc += 1
        # print(acc/(i+1))
        ratio_list.append('正确')
        print("第{i}/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    else:
        # print(acc/(i+1))
        ratio_list.append('错误')
        print("第{i}/100次测试, 儿童欺凌苗头诊断错误, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    
    index_list.append(i+1)
    # ratio_list.append(str(acc/(i+1) * 100) + '%')
    i += 1
print("测试完成, 100次儿童欺凌苗头诊断正确率: {ratio}%".format(i=i+1, ratio=acc))
ratio_list.append(str(acc/(i+1) * 100) + '%')
index_list.append('100次平均正确率')
stand_label.append('---')
sample_label.append('---')
# name = ['儿童心理问题', '准确率']
name = ['测试轮数', '样本标签', '预测标签', '识别结果']

index_list = np.array(index_list)
ratio_list = np.array(ratio_list)
face_landmarks = np.stack((index_list, stand_label, sample_label, ratio_list), axis=1)
landmarks = pd.DataFrame(columns=name, data=face_landmarks)
print("测试完成，已生成测试报告")
landmarks.to_csv('./儿童欺凌苗头功能测试报告.csv', encoding='gbk')

第1/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:100.0%
第2/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:100.0%
第3/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:100.0%
第4/100次测试, 儿童欺凌苗头诊断错误, 当前准确率为:75.0%
第5/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:80.0%
第6/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:83.33333333333334%
第7/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:85.71428571428571%
第8/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:87.5%
第9/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:88.88888888888889%
第10/100次测试, 儿童欺凌苗头诊断错误, 当前准确率为:80.0%
第11/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:81.81818181818183%
第12/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:83.33333333333334%
第13/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:84.61538461538461%
第14/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:85.71428571428571%
第15/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:86.66666666666667%
第16/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:87.5%
第17/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:88.23529411764706%
第18/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:88.88888888888889%
第19/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:89.47368421052632%
第20/100次测试, 儿童欺凌苗头诊断错误, 当前准确率为:85.0%
第21/100次测试, 儿童欺凌苗头诊断错误, 当前准确率为:80.95238095238095%
第22/100次测试, 儿童欺凌苗头诊断正确, 当前准确率为:81.81818181818183%
第23/100次测试, 儿童欺凌苗头诊断正确,

In [38]:
index_list = []
ratio_list = []
stand_label = []
sample_label = []

label_path = './muldataset/label.csv'
df = pd.read_csv(label_path)
df_mal = df['maladjustment'].values.tolist()
labels = df_mal

face = np.array(face)
labels = np.array(labels)
loo = LeaveOneOut()
i = 0
acc = 0
for train_index, test_index in loo.split(face):
    if i == 100:
        break
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = face[train_index], face[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    RF_clf = RandomForestClassifier(random_state=0, class_weight='balanced')
    RF_clf.fit(x_train, y_train)
    score_RF = RF_clf.score(x_test, y_test)
    y_predicted = RF_clf.predict(x_test)
    if y_test == 0:
        stand_label.append('正常')
    if y_test == 1:
        stand_label.append('异常')

    if y_predicted == 0:
        sample_label.append('正常')
    if y_predicted == 1:
        sample_label.append('异常')
        
    if y_predicted == y_test:
        acc += 1
        # print(acc/(i+1))
        ratio_list.append('正确')
        print("第{i}/100次测试, 儿童致病致残诊断正确, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    else:
        # print(acc/(i+1))
        ratio_list.append('错误')
        print("第{i}/100次测试, 儿童致病致残诊断错误, 当前准确率为:{acc}%".format(i=i+1, acc=acc/(i+1) * 100))
    
    index_list.append(i+1)
    # ratio_list.append(str(acc/(i+1) * 100) + '%')
    i += 1
print("测试完成, 100次儿童致病致残诊断正确率: {ratio}%".format(i=i+1, ratio=acc))
ratio_list.append(str(acc/(i+1) * 100) + '%')
index_list.append('100次平均正确率')
stand_label.append('---')
sample_label.append('---')
# name = ['儿童心理问题', '准确率']
name = ['测试轮数', '样本标签', '预测标签', '识别结果']

index_list = np.array(index_list)
ratio_list = np.array(ratio_list)
face_landmarks = np.stack((index_list, stand_label, sample_label, ratio_list), axis=1)
landmarks = pd.DataFrame(columns=name, data=face_landmarks)
print("测试完成，已生成测试报告")
landmarks.to_csv('./儿童心理疾病致残功能测试报告.csv', encoding='gbk')

第1/100次测试, 儿童致病致残诊断正确, 当前准确率为:100.0%
第2/100次测试, 儿童致病致残诊断正确, 当前准确率为:100.0%
第3/100次测试, 儿童致病致残诊断正确, 当前准确率为:100.0%
第4/100次测试, 儿童致病致残诊断错误, 当前准确率为:75.0%
第5/100次测试, 儿童致病致残诊断正确, 当前准确率为:80.0%
第6/100次测试, 儿童致病致残诊断正确, 当前准确率为:83.33333333333334%
第7/100次测试, 儿童致病致残诊断正确, 当前准确率为:85.71428571428571%
第8/100次测试, 儿童致病致残诊断错误, 当前准确率为:75.0%
第9/100次测试, 儿童致病致残诊断正确, 当前准确率为:77.77777777777779%
第10/100次测试, 儿童致病致残诊断错误, 当前准确率为:70.0%
第11/100次测试, 儿童致病致残诊断正确, 当前准确率为:72.72727272727273%
第12/100次测试, 儿童致病致残诊断正确, 当前准确率为:75.0%
第13/100次测试, 儿童致病致残诊断正确, 当前准确率为:76.92307692307693%
第14/100次测试, 儿童致病致残诊断正确, 当前准确率为:78.57142857142857%
第15/100次测试, 儿童致病致残诊断错误, 当前准确率为:73.33333333333333%
第16/100次测试, 儿童致病致残诊断正确, 当前准确率为:75.0%
第17/100次测试, 儿童致病致残诊断正确, 当前准确率为:76.47058823529412%
第18/100次测试, 儿童致病致残诊断正确, 当前准确率为:77.77777777777779%
第19/100次测试, 儿童致病致残诊断正确, 当前准确率为:78.94736842105263%
第20/100次测试, 儿童致病致残诊断错误, 当前准确率为:75.0%
第21/100次测试, 儿童致病致残诊断正确, 当前准确率为:76.19047619047619%
第22/100次测试, 儿童致病致残诊断正确, 当前准确率为:77.27272727272727%
第23/100次测试, 儿童致病致残诊断正确, 当前准确率为:78.26